In [6]:
import pandas as pd
import numpy as np
import random
    
uk= pd.read_csv(r'C:\Users\cti110016\Dropbox\DB\retail_clean.csv')
uk['InvoiceDate']= pd.to_datetime(uk['InvoiceDate'])
uk['date']= uk['InvoiceDate'].dt.strftime('%Y-%m-%d')
uk['date']= pd.to_datetime(uk['date'])


uk.columns

a=uk.groupby(['date','Invoice']).agg(n_invoices= ('Invoice','count')).reset_index().groupby('date').agg(n_invoices=('Invoice','count'))
uk_weekly= a.resample('W').sum().reset_index()

#1)	Which week year has the highest conversion rate ?

footfall_uk= pd.read_excel(r'C:\Users\cti110016\Dropbox\DB\footfall_uk.xlsx')
footfall_uk['date']=pd.to_datetime(footfall_uk['date'])

footfall_uk.set_index('date',inplace=True)
footfall_weekly= footfall_uk.resample('W')['website_visitors'].sum()
footfall_weekly=footfall_weekly.reset_index()
joined= pd.merge(uk_weekly,footfall_weekly,how='left')
joined['conversion_rate']= joined['n_invoices']/joined['website_visitors']

joined[joined.conversion_rate== max(joined.conversion_rate)]

#2)	Which week year has the highest average transaction value ?

uk['revenue']= uk['Price']* uk['Quantity']

revenue_per_invoice= uk.groupby(['date','Invoice']).agg(revenue=('revenue','sum'),
                                                        Qty= ('Quantity','sum')).reset_index()

revenue_per_invoice['week']=revenue_per_invoice['date'].dt.isocalendar().week
revenue_per_invoice['year']=revenue_per_invoice['date'].dt.isocalendar().year

ATV= revenue_per_invoice.groupby(['year','week']).agg(date= ('date','first'),
                                                                             ATV= ('revenue','mean'),
                                                                             UPT= ('Qty','mean'),
                                                                             n_invoices= ('Invoice','count')).reset_index()
ATV[ATV.ATV== max(ATV.ATV)]

#3)	Which week year has the lowest units per transaction?
ATV[ATV.UPT== min(ATV.UPT)]


#4)	Which week has the lowest average selling price ?

ATV['ASP']=ATV['ATV']/ATV['UPT']
ATV[ATV.ASP== min(ATV.ASP)]

,year,week,date,ATV,UPT,n_invoices,ASP
35,2010,32,2010-08-09,534.523205,541.439169,337,0.987227
